###  Mret xlsx to Tidydata

In [1]:
from databaker.framework import *
import pandas as pd

In [2]:
 import requests
from pathlib import Path

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/tradeingoodsmretsallbopeu2013timeseriesspreadsheet/current/mret.xlsx'
inputFile = sourceFolder / 'mret.xls'
if not(inputFile.exists() and inputFile.is_file()):
    response = requests.get(inputURL)
    with open(inputFile, 'wb') as f:
        f.write(response.content)

In [ ]:
# inputFile = 'mret.xlsx'

In [3]:
tab = loadxlstabs(inputFile)

Loading in\mret.xls which has size 2795578 bytes
Table names: ['data']


In [4]:
tab = tab[0]

In [5]:
observations = tab.excel_ref('B8').expand(DOWN).expand(RIGHT).is_not_blank()

In [6]:
Title = tab.excel_ref('B1').expand(RIGHT).is_not_blank()

In [7]:
CDID = tab.excel_ref('B2').expand(RIGHT).is_not_blank()

In [8]:
Year = tab.excel_ref('A8').expand(DOWN).is_not_blank()

In [9]:
Currency = tab.excel_ref('P3')

In [10]:
Dimensions = [
            HDimConst('Geography', 'K02000001'),
            HDim(Year,'TIME',DIRECTLY,LEFT),
            HDim(CDID,'Cdid',DIRECTLY,ABOVE),
            HDimConst('Unit', '£ Millions'), 
            HDimConst('Measure Type','GBP Total'),            
            HDim(Title,'Title',DIRECTLY,ABOVE)
    
]

In [11]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)

In [12]:
new_table = c1.topandas()

multiple TIMEUNITs: 'Year'(15790), 'Quarter'(63160), ''(190162)


In [ ]:
# new_table['Title'] = new_table['Title'].map(lambda cell:cell.replace('£m', ''))

In [ ]:
# new_table = new_table.drop('TIMEUNIT', axis=1)

In [13]:
print(len(new_table))

269112


In [ ]:
# new_table.fillna('', inplace=True)

# writetechnicalCSV('C:\\Users\\vamsi\\Desktop\\Gdrive\\Out\\Mret\\mretWDA.csv', new_table)

In [14]:
temp_table = pd.read_excel('cord_sitc classification table.xlsx', sheet_name = 0)
temp_table = temp_table.reset_index()
temp_table.drop(['level_0','level_1','Sequence'], axis =1, inplace = True)
temp_table.tail(5)


,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
6196,BPFR,5min8minE,WW,IM,BOP,IDEF,NSA,M
6197,BPGR,5min8minE,WW,IM,BOP,IDEF,SA,M
6198,BPCR,5min8minE,WW,IM,BOP,VM,NSA,M
6199,ELAI,5min8minE,WW,IM,BOP,VM,SA,M
6200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
classification1 = pd.read_excel('CSDB classification tables.xlsx', 'cord_sitc')
classification2 = pd.read_excel('CSDB classification tables.xlsx', 'cord_cpa')
classification3 = pd.read_excel('CSDB classification tables.xlsx', 'cord_country')
classification4 = pd.read_csv('Codelist.csv')

In [19]:
classification1.head(5)

,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
0,SDSX,2plus4,EU,BAL,BOP,CP,NSA,Q
1,SGLO,5minus8,EU,BAL,BOP,CP,NSA,Q
2,SESL,5plus6,EU,BAL,BOP,CP,NSA,Q
3,SFJC,7plus8,EU,BAL,BOP,CP,NSA,Q
4,LKTX,TminusO,EU,BAL,BOP,CP,NSA,Q


In [20]:
classification2.head(5)

,cdid,PRODUCT,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
0,P42L,24.2,EU,EX,BOP,CP,NSA,Q
1,P483,24.2,EU,EX,BOP,CP,SA,Q
2,P4DJ,24.2,EU,EX,BOP,CVM,NSA,Q
3,P4IZ,24.2,EU,EX,BOP,CVM,SA,Q
4,P3EP,24.2,EU,IM,BOP,CP,NSA,Q


In [21]:
classification3.head(5)

,cdid,COUNTRY,DIRECTION,BASIS,SEASADJ,PERIOD
0,KN2O,XS,BAL,BOP,NSA,Q
1,LGDS,V4,BAL,BOP,NSA,Q
2,L87P,V3,BAL,BOP,NSA,Q
3,L87J,V2,BAL,BOP,NSA,Q
4,MHN8,I7,BAL,BOP,NSA,Q


In [18]:
classification4.head(5)

,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
0,AJFB,NaN,UK,NaN,BE,CP,NSA,NaN
1,AJFD,NaN,UK,NaN,BE,CP,NSA,NaN
2,AJFI,NaN,UK,NaN,BE,CP,NSA,NaN
3,AJFJ,NaN,UK,NaN,BE,CP,NSA,NaN
4,AJFK,NaN,UK,NaN,BE,CP,NSA,NaN


In [22]:
classification2.rename(index = str, columns = {'PRODUCT':'COMMODITY'}, inplace = True)

In [23]:
classification3['COMMODITY'] = ''
classification3['PRICE'] = ''

In [24]:
classification3.rename(index = str, columns = {'COUNTRY':'AREA'}, inplace = True)

In [25]:
temp_table = pd.concat([temp_table, classification1, classification2,classification3,classification4])

In [26]:
temp_table.head()

,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,EU,BOP,2plus4,BAL,M,CP,NSA,SDSX
1,EU,BOP,5minus8,BAL,M,CP,NSA,SGLO
2,EU,BOP,7plus8,BAL,M,CP,NSA,SFJC
3,EU,BOP,TminusO,BAL,M,CP,NSA,LKTX
4,EU,BOP,0plus1,BAL,M,CP,NSA,SDMS


In [27]:
temp_table.shape

(18295, 8)

In [28]:
temp_table.drop_duplicates(['cdid'], keep='first', inplace=True)

In [29]:
temp_table.head()

,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,EU,BOP,2plus4,BAL,M,CP,NSA,SDSX
1,EU,BOP,5minus8,BAL,M,CP,NSA,SGLO
2,EU,BOP,7plus8,BAL,M,CP,NSA,SFJC
3,EU,BOP,TminusO,BAL,M,CP,NSA,LKTX
4,EU,BOP,0plus1,BAL,M,CP,NSA,SDMS


In [30]:
new_table.head(5)

,OBS,TIME,TIMEUNIT,Geography,Cdid,Unit,Measure Type,Title
0,1039.0,1955,Year,K02000001,IKBB,£ Millions,GBP Total,BOP: Exports:CP SA: Total Trade in Services £m
1,997.0,1955,Year,K02000001,IKBC,£ Millions,GBP Total,BOP:Imports:CP SA:Total Trade in Services £m
2,1147.0,1956,Year,K02000001,IKBB,£ Millions,GBP Total,BOP: Exports:CP SA: Total Trade in Services £m
3,1121.0,1956,Year,K02000001,IKBC,£ Millions,GBP Total,BOP:Imports:CP SA:Total Trade in Services £m
4,1250.0,1957,Year,K02000001,IKBB,£ Millions,GBP Total,BOP: Exports:CP SA: Total Trade in Services £m


In [31]:
new_table.head(1)['Cdid']

0    IKBB
Name: Cdid, dtype: object

In [32]:
temp_table.head(1)['cdid']

0    SDSX
Name: cdid, dtype: object

In [33]:
new_table = pd.merge(new_table,temp_table, how = 'left', left_on = 'Cdid', right_on = 'cdid')

In [34]:
new_table.head(5)

,OBS,TIME,TIMEUNIT,Geography,Cdid,Unit,Measure Type,Title,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,1039.0,1955,Year,K02000001,IKBB,£ Millions,GBP Total,BOP: Exports:CP SA: Total Trade in Services £m,NaN,NaN,NaN,EX,NaN,CP,SA,IKBB
1,997.0,1955,Year,K02000001,IKBC,£ Millions,GBP Total,BOP:Imports:CP SA:Total Trade in Services £m,NaN,NaN,NaN,IM,NaN,CP,SA,IKBC
2,1147.0,1956,Year,K02000001,IKBB,£ Millions,GBP Total,BOP: Exports:CP SA: Total Trade in Services £m,NaN,NaN,NaN,EX,NaN,CP,SA,IKBB
3,1121.0,1956,Year,K02000001,IKBC,£ Millions,GBP Total,BOP:Imports:CP SA:Total Trade in Services £m,NaN,NaN,NaN,IM,NaN,CP,SA,IKBC
4,1250.0,1957,Year,K02000001,IKBB,£ Millions,GBP Total,BOP: Exports:CP SA: Total Trade in Services £m,NaN,NaN,NaN,EX,NaN,CP,SA,IKBB


In [35]:
new_table.drop(['cdid'], axis = 1, inplace = True)

In [36]:
new_table['Year'] = new_table['TIME'].map(lambda x: str(x)[:4])

In [37]:
new_table['Months/Quarter'] = new_table['TIME'].map(lambda x: str(x)[4:])

In [39]:
new_table = new_table.drop('TIME', axis=1)

In [40]:
new_table.rename(index= str, columns= {'Year':'TIME'}, inplace = True)

In [43]:
new_table['Months/Quarter'] = new_table['Months/Quarter'].str.lstrip('')

In [44]:
new_table['Period'] = new_table['TIMEUNIT'].map(str)+'/'+new_table['TIME']+'-'+new_table['Months/Quarter']

In [45]:
new_table['Period'] = new_table['Period'].str.rstrip('-')

In [46]:
new_table['Period'] = new_table['Period'].map(lambda cell:cell.replace(' ', ''))

In [49]:
new_table['Period'] = new_table['Period'].map(lambda cell:cell.replace('Year', 'year'))

In [61]:
new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)
new_table.rename(index= str, columns= {'DIRECTION':'Flow'}, inplace = True)
new_table.rename(index= str, columns= {'COMMODITY':'Product'}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [53]:
new_table.drop(['TIME'], axis = 1, inplace = True)
new_table.drop(['TIMEUNIT'], axis = 1, inplace = True)
new_table.drop(['Geography'], axis = 1, inplace = True)
new_table.drop(['PERIOD'], axis = 1, inplace = True)
new_table.drop(['Months/Quarter'], axis = 1, inplace = True)


In [62]:
new_table =new_table[['AREA','Period','Cdid','BASIS','Product','SEASADJ','Flow','PRICE','Measure Type','Value','Unit']]

In [63]:
new_table.tail(5)

,AREA,Period,Cdid,BASIS,Product,SEASADJ,Flow,PRICE,Measure Type,Value,Unit
269107,WW,/2018-FEB,QALU,BOP,792,SA,EX,CP,GBP Total,1282.0,£ Millions
269108,RW,/2018-FEB,SGRX,BOP,9,SA,EX,CP,GBP Total,457.0,£ Millions
269109,WW,/2018-FEB,QALW,BOP,792,SA,BAL,CP,GBP Total,757.0,£ Millions
269110,WW,/2018-FEB,QALV,BOP,792,SA,IM,CP,GBP Total,525.0,£ Millions
269111,RW,/2018-FEB,SGTK,BOP,9,SA,IM,CP,GBP Total,439.0,£ Millions


Pull out missing Values

In [ ]:
# temp_table1 = new_table[new_table['AREA'].isnull() == True] 
# Codelist = temp_table1['Cdid'].unique()
# Codelist.shape
# codes = pd.Series(Codelist)
# codes.to_csv('Codelist.csv', index = False)

In [65]:
new_table.dtypes

AREA             object
Period           object
Cdid             object
BASIS            object
Product          object
SEASADJ          object
Flow             object
PRICE            object
Measure Type     object
Value           float64
Unit             object
dtype: object

In [66]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('MRET_Tidydata.csv'), index = False)

In [64]:
# new_table.to_csv('C:\\Users\\vamsi\\Desktop\\Gdrive\\Out\\Mret\\MRET_Tidydata.csv', index = False)